In [ ]:
# MAE = 56
import tensorflow as tf
from tensorflow.keras import layers, models

def dilated_conv_block(x, filters, dilation_rates):
    convs = []
    for rate in dilation_rates:
        conv = layers.Conv2D(filters, 3, padding='same', dilation_rate=rate, activation='relu')(x)
        convs.append(conv)
    return layers.Concatenate()(convs)

def multi_scale_conv_block(x, filters):
    conv3 = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    conv5 = layers.Conv2D(filters, 5, padding='same', activation='relu')(x)
    conv7 = layers.Conv2D(filters, 7, padding='same', activation='relu')(x)
    return layers.Concatenate()([conv3, conv5, conv7])

In [6]:
def scale_aggregation_module(x, filters):
    x = layers.Conv2D(filters, 1, padding='same', activation='relu')(x)
    conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    conv2 = layers.Conv2D(filters, 3, padding='same', dilation_rate=2, activation='relu')(x)
    conv3 = layers.Conv2D(filters, 3, padding='same', dilation_rate=3, activation='relu')(x)
    x = layers.Concatenate()([conv1, conv2, conv3])
    x = layers.Conv2D(filters, 1, padding='same', activation='relu')(x)
    return x

def build_backbone(input_shape=(1024, 768, 3)):
    vgg16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in vgg16.layers:
        layer.trainable = False
    block3_out = vgg16.get_layer('block3_conv3').output
    return models.Model(inputs=vgg16.input, outputs=block3_out, name='vgg16_block3')

In [7]:
def build_crowd_counting_model(input_shape=(1024, 768, 3)):
    inputs = layers.Input(shape=input_shape)
    backbone = build_backbone(input_shape)
    x = backbone(inputs) 

    x = multi_scale_conv_block(x, filters=64) 
    x = dilated_conv_block(x, filters=64, dilation_rates=[1, 2, 3])
    x = scale_aggregation_module(x, filters=128) 

    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    density_map = layers.Conv2D(1, 1, padding='same', activation='linear')(x)  

    model = models.Model(inputs=inputs, outputs=density_map, name='CrowdCountNet')
    return model

model = build_crowd_counting_model()
model.summary()

Model: "CrowdCountNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 1024, 768, │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16_block3        │ (None, 256, 192,  │  1,735,488 │ input_layer_2[0]… │
│ (Functional)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 256, 192,  │    147,520 │ vgg16_block3[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 256, 192,  │    409,664 │ vgg16_block3[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 256, 192,  │    802,880 │ vgg16_block3[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 256, 192,  │          0 │ conv2d_14[0][0],  │
│ (Concatenate)       │ 192)              │            │ conv2d_15[0][0],  │
│                     │                   │            │ conv2d_16[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 256, 192,  │    110,656 │ concatenate_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 256, 192,  │    110,656 │ concatenate_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 256, 192,  │    110,656 │ concatenate_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 256, 192,  │          0 │ conv2d_17[0][0],  │
│ (Concatenate)       │ 192)              │            │ conv2d_18[0][0],  │
│                     │                   │            │ conv2d_19[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 256, 192,  │     24,704 │ concatenate_4[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 256, 192,  │    147,584 │ conv2d_20[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 256, 192,  │    147,584 │ conv2d_20[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 256, 192,  │    147,584 │ conv2d_20[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 256, 192,  │          0 │ conv2d_21[0][0],  │
│ (Concatenate)       │ 384)              │            │ conv2d_22[0][0],  │
│                     │                   │            │ conv2d_23[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 256, 192,  │     49,280 │ concatenate_5[0]

 Total params: 4,036,545 (15.40 MB)

 Trainable params: 2,301,057 (8.78 MB)

 Non-trainable params: 1,735,488 (6.62 MB)